In [ ]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

def preprocess_image(image):
    lab_image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l_channel, a_channel, b_channel = cv2.split(lab_image)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    clahe_l_channel = clahe.apply(l_channel)
    enhanced_lab_image = cv2.merge((clahe_l_channel, a_channel, b_channel))
    enhanced_bgr_image = cv2.cvtColor(enhanced_lab_image, cv2.COLOR_LAB2BGR)
    blurred_image = cv2.GaussianBlur(enhanced_bgr_image, (5, 5), 0)
    return blurred_image

def segment_blood_cells(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)
    _, segmented_cells = cv2.threshold(blurred_image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    kernel = np.ones((3, 3), np.uint8)
    segmented_cells = cv2.morphologyEx(segmented_cells, cv2.MORPH_OPEN, kernel, iterations=2)
    segmented_cells = cv2.morphologyEx(segmented_cells, cv2.MORPH_CLOSE, kernel, iterations=2)
    return segmented_cells

def extract_features(image):
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    features = []
    for contour in contours:
        area = cv2.contourArea(contour)
        perimeter = cv2.arcLength(contour, True)
        features.append([area, perimeter])
    features = np.array(features)
    return features

def load_data(images_path, labels_path):
    images = []
    labels = []
    for filename in os.listdir(images_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  
            image_path = os.path.join(images_path, filename)
            label_path = os.path.join(labels_path, filename.split('.')[0] + '.txt')  
            if os.path.exists(label_path):
                image = cv2.imread(image_path)
                if image is not None:
                    preprocessed_image = preprocess_image(image)
                    with open(label_path, 'r') as f:
                        label = f.read().strip()
                    images.append(preprocessed_image)
                    labels.append(label)
                else:
                    print("Failed to load image:", image_path)
            else:
                print("Label file not found for:", image_path)
    if len(images) == 0 or len(labels) == 0:
        raise ValueError("No data loaded. Check the paths or file formats.")
    return images, labels


def main():
    train_path = '/Users/mohammadadnaan/Desktop/Projects/Blood Group Detection/train'
    test_path = '/Users/mohammadadnaan/Desktop/Projects/Blood Group Detection/test'
    val_path = '/Users/mohammadadnaan/Desktop/Projects/Blood Group Detection/valid'
    
    try:
        X_train, y_train = load_data(os.path.join(train_path, 'images'), os.path.join(train_path, 'labels'))
        X_test, y_test = load_data(os.path.join(test_path, 'images'), os.path.join(test_path, 'labels'))
        X_val, y_val = load_data(os.path.join(val_path, 'images'), os.path.join(val_path, 'labels'))

        print("Number of training samples:", len(X_train))
        print("Number of testing samples:", len(X_test))
        print("Number of validation samples:", len(X_val))

        compare_classifiers(X_train, y_train, X_val, y_val)
    except Exception as e:
        print("An error occurred:", str(e))

def compare_classifiers(X_train, y_train, X_val, y_val):
    svm_classifier = train_svm(X_train, y_train)
    svm_accuracy = evaluate_classifier(svm_classifier, X_val, y_val)
    print("SVM Accuracy:", svm_accuracy)

    knn_classifier = train_knn(X_train, y_train)
    knn_accuracy = evaluate_classifier(knn_classifier, X_val, y_val)
    print("KNN Accuracy:", knn_accuracy)

    decision_tree_classifier = train_decision_tree(X_train, y_train)
    decision_tree_accuracy = evaluate_classifier(decision_tree_classifier, X_val, y_val)
    print("Decision Tree Accuracy:", decision_tree_accuracy)

    ann_classifier = train_ann(X_train, y_train)
    ann_accuracy = evaluate_classifier(ann_classifier, X_val, y_val)
    print("ANN Accuracy:", ann_accuracy)

def train_svm(X_train, y_train):
    clf = SVC(kernel='linear')
    clf.fit(X_train, y_train)
    return clf

def train_knn(X_train, y_train):
    clf = KNeighborsClassifier(n_neighbors=5)
    clf.fit(X_train, y_train)
    return clf

def train_decision_tree(X_train, y_train):
    clf = DecisionTreeClassifier()
    clf.fit(X_train, y_train)
    return clf

def train_ann(X_train, y_train):
    clf = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500)
    clf.fit(X_train, y_train)
    return clf

def evaluate_classifier(clf, X_val, y_val):
    y_pred = clf.predict(X_val)
    accuracy = accuracy_score(y_val, y_pred)
    return accuracy

if __name__ == "__main__":
    main()
